Goal of this script is to obtain 
vertex, edge, face 


V, EV

In [108]:
pwd()


"/home/mjirik/projects/lario3d.jl/notebooks"

In [109]:
# hexaogon
#    3 -- 4
#  /  \ /  \
# 1 -- 2 -- 5
#  \  / \  /
#   7 -- 8
hexagon_vertices = [0. 0.; 2. 0.; 1. 2.; 3. 2; 4. 0.; 3. -2.; 1. -2]
hexagon_faces = [ 1 2 3; 2 4 3; 4 2 5; 5 2 6; 2 7 6; 2 1 7]
# maximum(test_faces0)



6×3 Array{Int64,2}:
 1  2  3
 2  4  3
 4  2  5
 5  2  6
 2  7  6
 2  1  7

# Read obj file

In [208]:

include("../src/plasm.jl")
include("../src/read.jl")

to_lar (generic function with 1 method)

In [110]:

function get_obj_metainfo(lines)
    """
    lines = readlines(open(filename))
    """
    
    nvertices = 0
    nfaces = 0
    for line in lines
        if line[1] == 'v'
            nvertices = nvertices + 1
#             print("v ", nvertices)
        elseif line[1] == 'f'
            nfaces = nfaces + 1
#             print("f ", nfaces, " ")
        end
#         print(line)
    end
    return nvertices, nfaces
end

function get_obj_vertices_and_faces(lines, nvertices, nfaces)
    vertices = Array(Float64, nvertices , 3)
    faces = Array(Int64, nfaces, 3)
#     println("vertices ", size(vertices))
#     println("faces ", size(vertices))
    ivertices = 1
    ifaces = 1

    for line in lines
#         println("ivertices and ifaces ", ivertices, " ",ifaces)
        lsplit = split(line)
        if line[1] == 'v'
            for i=2:4
                vertices[ivertices, i - 1] = parse(Float64, lsplit[i])
            end
    #         print(vertices[ivertices,:])
    #         println(line)
    #         println(vertices[1:10,:])

            ivertices = ivertices + 1
        elseif line[1] == 'f'
            for i=2:4
                faces[ifaces, i - 1] = parse(Int64, lsplit[i])
            end
            ifaces = ifaces + 1
        else
            println("Cannot parse: ", line)
        end
    end
    return vertices, faces
end

function read_obj(filename)
    
    f = open(filename);
    lines = readlines(f)
    nvertices, nfaces = get_obj_metainfo(lines)
#     println(nvertices, " ", nfaces)
    vertices, faces = get_obj_vertices_and_faces(lines, nvertices, nfaces)
#     println(nvertices, " ", nfaces)
    return vertices, faces
end

# TODO delete
# filename = "../test/smallbb2.obj"
# vertices, faces = read_obj(filename)


read_obj (generic function with 1 method)

In [197]:
function check_vf(vertices, faces, display=false)
    """
    Analyse limits of vertexes and faces
    """

    mn = minimum(faces)
    mx = maximum(faces)
    vn = size(vertices, 1)
    if mn < 1
        println("vertex index lower than 1")
        println("faces min $(mn), faces max $(mx), vertex number $(vn)")
    end
    if mx < vn
        println("vertex index index higher than voxel number")
        println("faces min $(mn), faces max $(mx), vertex number $(vn)")
    end
    if display
        println("faces min $(mn), faces max $(mx), vertex number $(vn)")
    end
end

    

check_vf (generic function with 2 methods)

# Check Julia

## Vertices

In [112]:
V = vertices

1192×3 Array{Float64,2}:
 1.0  2.0  1.0
 2.0  2.0  1.0
 1.0  3.0  1.0
 2.0  3.0  1.0
 1.0  2.0  1.0
 2.0  2.0  1.0
 1.0  2.0  2.0
 2.0  2.0  2.0
 1.0  2.0  1.0
 1.0  3.0  1.0
 1.0  2.0  2.0
 1.0  3.0  2.0
 2.0  2.0  1.0
 ⋮            
 4.0  6.0  5.0
 5.0  6.0  5.0
 4.0  6.0  6.0
 5.0  6.0  6.0
 4.0  4.0  6.0
 5.0  4.0  6.0
 4.0  5.0  6.0
 5.0  5.0  6.0
 4.0  5.0  6.0
 5.0  5.0  6.0
 4.0  6.0  6.0
 5.0  6.0  6.0

## EV naive

In [113]:
function get_EVinds_naive(faces, nfaces=nothing)
    if nfaces == nothing
        nfaces = size(faces,1)
    end
    # Eset = Set()
    Einds = []
    EVinds = []
    FEinds = Array(Int64, nfaces, 3)

    for i = 1:nfaces

        push!(EVinds, sort([faces[i, 1], faces[i, 2]]))
        push!(EVinds, sort([faces[i, 2], faces[i, 3]]))
        push!(EVinds, sort([faces[i, 3], faces[i, 1]]))

        FEinds[i, :] = [(i-1)*3 + 1, (i-1)*3 + 2,(i-1)*3 + 3]

    end

    return EVinds
end
# Epl = EVpl


get_EVinds_naive (generic function with 2 methods)

In [114]:
# tohle bude patrně blbost
# nzind2sparse(nzind)
function get_FElar_naive(nfaces)

    FElar = spzeros(Int8, nfaces, (3*(nfaces)))
    for i = 1:nfaces
        FElar[i, (i-1)*3 + 1] = 1
        FElar[i, (i-1)*3 + 2] = 1
        FElar[i, (i-1)*3 + 3] = 1
    end
    return FElar
end


get_FElar_naive (generic function with 3 methods)

# EV no double edges

In [115]:
faces = hexagon_faces
nfaces = size(hexagon_faces, 1)

6

In [116]:
function bool2sgn(bl)
    if (bl) 
        a1 = 1
    else
        a1 = -1
    end
    
    return a1
end

function face_orientation(face)
    a1 = face[1] > face[2] 
    a2 = face[2] > face[3] 
    a3 = face[3] > face[1]
    signum = bool2sgn(a1) * bool2sgn(a2) * bool2sgn(a3)
    return signum
end
    
#test
assert(1 == face_orientation([1 2 3]))
assert(-1 == face_orientation([1 3 2]))

In [117]:

# function get_EVinds(faces, nfaces=nothing)
#     if nfaces == nothing
#         nfaces = size(faces,1)
#     end
    # Eset = Set()

function iter_edge(edge, EVinv, iedges)
    """
    Add edge to EVinds_dict if it is not there end return edge_id.
    """
    # edge1
    edge_tuple = tuple(edge[:]...)
    if haskey(EVinv, edge_tuple)
        edge_id = EVinv[edge_tuple]
    else
        iedges = iedges + 1
        EVinv[edge_tuple] = iedges
        edge_id = iedges
    end
    edge1id = edge_id
    
    return edge_id, edge_tuple, iedges
end

function EVinv_to_EVinds(EVinv, nedges)
    """
    EVinv: inverse edge to vertes. Dictionary where key is tuple with vertex index and value is edge id.
    """
    EVinds = Array(Int64, nedges, 2)
    for pair in EVinv
        edge, edgeid = pair
        EVinds[edgeid, 1] = edge[1]
        EVinds[edgeid, 2] = edge[2]
    end
    return EVinds
end

function get_FEinds_and_EVinv(faces, nfaces)
    """
    :faces: Array nfaces x 3
    
    Return
    FEinds: face to edge array. Contain indexes of edges
    EVinv: inverse edge to vertes. Dictionary where key is tuple with vertex index and value is edge id.
    """
# Create list of unique edges and create FEinds
# Keep order of vertex in faces. It is important for face orientation.
    Einds = []
    EVinv = Dict()
    FEinds = Array(Int64, nfaces, 3)
    iedges = 0
    for i = 1:nfaces
        face = sort(faces[i,:])

        # edge1
        edge = [face[1], face[2]]
        edge1_id, edge_tuple, iedges = iter_edge(edge, EVinv, iedges)

        edge = [face[2], face[3]]
        edge2_id, edge_tuple, iedges = iter_edge(edge, EVinv, iedges)

        edge = [face[1], face[3]]
        edge3_id, edge_tuple, iedges = iter_edge(edge, EVinv, iedges)

        FEinds[i, :] = [edge1_id, edge2_id, edge3_id]

    end
    nedges = iedges
    return FEinds, EVinv, nedges
end

function convert_to_FElar(faces, FEinds, nfaces, nedges)
    FElar = spzeros(Int8, nfaces, nedges)
    for i = 1:nfaces
        face_orient = face_orientation(faces[i,:])
#         println(face_orient)
        edges_inds = FEinds[i,:]
        FElar[i, edges_inds[1]] = face_orient * 1
        FElar[i, edges_inds[2]] = face_orient * 1
        FElar[i, edges_inds[3]] = face_orient * -1

        #     face_orient = face_orientation(faces[i,:])
        #     face = sort(faces[i,:])
        #     edge = [face[1], face[2]]
        #     edge_tuple = tuple(edge[:]...)  
    end
    return FElar

end


function get_FEinds_and_EVinds(faces, nfaces=nothing)
    if nfaces == nothing
        nfaces = size(faces,1)
    end
    FEinds, EVinv, nedges = get_FEinds_and_EVinv(faces, nfaces)
    EVinds = EVinv_to_EVinds(EVinv, nedges)
    return FEinds, EVinds
end

function convert_to_EVlar(EVinds, nedges, nvertices)
    EVlar = spzeros(Int8, nedges, nvertices)
    for i in 1:size(EVlar, 1)
        EVlar[i, EVinds[i, 1]] = -1
        EVlar[i, EVinds[i, 2]] = 1
    end
    return EVlar
end
#     FElar = get_FElar(faces, FEinds, nfaces, nedges)
#     return EVinds
# end

function get_EVlar_FElar(vertices, faces)
    FEinds, EVinds = get_FEinds_and_EVinds(faces)
    nedges = size(EVinds, 1)
    nfaces = size(faces, 1)
    nvertices = size(vertices, 1)
    FElar = convert_to_FElar(faces, FEinds, nfaces, nedges)
#     FElar = convert_to_FElar(faces, FEinds, nfaces, nedges)
    EVlar = convert_to_EVlar(EVinds, nedges, nvertices)
    
    return EVlar, FElar
end

function to_lar(vertices, faces)
    EVlar, FElar = get_EVlar_FElar(vertices, faces)
return vertices, EVlar, FElar
    

to_lar (generic function with 1 method)

In [118]:
# get_FEinds_and_EVinv(hexagon_faces, size(hexagon_faces, 1))

In [119]:

# test
FEinds, EVinds = get_FEinds_and_EVinds(hexagon_faces)
assert(size(FEinds,1) == 6)
assert(size(FEinds,2) == 3)
assert(size(EVinds,1) == 12)
assert(size(EVinds,2) == 2)
# full(EVinds)


# TODO FElar EVlar

In [120]:
nedges = size(EVinds, 1)
nfaces = size(hexagon_faces, 1)
nvertices = size(hexagon_vertices, 1)

7

In [121]:
FElar = convert_to_FElar(hexagon_faces, FEinds, nfaces, nedges)
full(FElar)

6×12 Array{Int8,2}:
  1   1  -1   0   0   0   0   0   0   0   0  0
  0  -1   0  -1   1   0   0   0   0   0   0  0
  0   0   0   0  -1  -1   1   0   0   0   0  0
  0   0   0   0   0   0  -1  -1   1   0   0  0
  0   0   0   0   0   0   0   0  -1  -1   1  0
 -1   0   0   0   0   0   0   0   0   0  -1  1

In [122]:

EVlar = convert_to_EVlar(EVinds, nedges, nvertices)

12×7 sparse matrix with 24 Int8 nonzero entries:
	[1 ,  1]  =  -1
	[3 ,  1]  =  -1
	[12,  1]  =  -1
	[1 ,  2]  =  1
	[2 ,  2]  =  -1
	[5 ,  2]  =  -1
	[7 ,  2]  =  -1
	[9 ,  2]  =  -1
	[11,  2]  =  -1
	[2 ,  3]  =  1
	⋮
	[5 ,  4]  =  1
	[6 ,  4]  =  -1
	[6 ,  5]  =  1
	[7 ,  5]  =  1
	[8 ,  5]  =  -1
	[8 ,  6]  =  1
	[9 ,  6]  =  1
	[10,  6]  =  -1
	[10,  7]  =  1
	[11,  7]  =  1
	[12,  7]  =  1

In [123]:
# Visualization 

In [183]:

include("../src/plasm.jl")

visualize_boundary (generic function with 1 method)

In [125]:
vf_limits(hexagon_vertices, hexagon_faces, true)

faces min 1, faces max 7, vertex number 7


In [126]:

visualize_numbers([hexagon_vertices, EVlar, FElar], 2)

1
3
4
5
6
2
XgeModule::init_started
Creating shared GLCanvas...
shared GLCanvas created
XgeModule::init_finished
Evaluating fenvs.py..
('...fenvs.py imported in', 0.007852000000000081, 'seconds')
centre
centre
centre
Building batches from Hpc....
...done in 2 msec
Optimizing the octree....
   Number of input batches 235
   total number vertices    130
   Number of output batches 65
   Batch vertex media       2
...done in 0 msec
Building octree from 65 batches....
Scene number of nodes of the octree 130
Scene max depth                     4
Scene number of batches             65
...done in 0 msec


![hexagon](https://raw.githubusercontent.com/mjirik/lario3d.jl/master/graphics/hexagon_lario3djl.png)

In [184]:
ev, fe = to_lar(hexagon_vertices, hexagon_faces)

visualize_numbers([hexagon_vertices, ev, fe], 2)

1
3
4
5
6
2
XgeModule::init_started
Creating shared GLCanvas...
shared GLCanvas created
XgeModule::init_finished
Evaluating fenvs.py..
('...fenvs.py imported in', 0.00825599999999993, 'seconds')
centre
centre
centre
Building batches from Hpc....
...done in 2 msec
Optimizing the octree....
   Number of input batches 235
   total number vertices    130
   Number of output batches 65
   Batch vertex media       2
...done in 0 msec
Building octree from 65 batches....
Scene number of nodes of the octree 130
Scene max depth                     4
Scene number of batches             65
...done in 0 msec


In [200]:
function check_vevfe(vertices, ev, fe)

    vnv = size(vertices, 1)
    fnfe = size(fe, 1)
    enfe = size(fe, 2)
    enev = size(ev, 1)
    vnev = size(ev, 2)
    

    if enfe != enev
        println("Different number of edges in EV and FE")
    end
    if vnv != vnev
        println("Different number of vertices in EV and V")
    end

    if -1 != minimum(fe)
        println("Minimum value of FE should be -1 and it is $(minimum(fe))")
    end
    if 1 != maximum(fe)
        println("Maximum value of FE should be 1 and it is $(maximum(fe))")
    end
    if -1 != minimum(ev)
        println("Minimum value of EV should be -1 and it is $(minimum(fe))")
    end
    if 1 != maximum(ev)
        println("Maximum value of EV should be 1 and it is $(maximum(fe))")
    end


    if eltype(fe) != Int8
        println("Element type of FE should be Int8 and it is $(eltype(fe))")
    end

    if eltype(ev) != Int8
        println("Element type of EV should be Int8 and it is $(eltype(ev))")
    end
    
end


# vn = size(vertices, 1)
# if mn < 1
#     println("vertex index lower than 1")
#     println("faces min $(mn), faces max $(mx), vertex number $(vn)")
# end

check_vevfe (generic function with 1 method)

In [201]:
full(fe)

6×12 Array{Int8,2}:
  1   1  -1   0   0   0   0   0   0   0   0  0
  0  -1   0  -1   1   0   0   0   0   0   0  0
  0   0   0   0  -1  -1   1   0   0   0   0  0
  0   0   0   0   0   0  -1  -1   1   0   0  0
  0   0   0   0   0   0   0   0  -1  -1   1  0
 -1   0   0   0   0   0   0   0   0   0  -1  1

In [202]:

filename = "../test/smallbb2.obj"
vertices, faces = read_obj(filename)
nvertices = size(vertices,1)
nfaces = size(faces,1)

# assert(nvertices == 1192)
# assert(nfaces == 298)
# println(nfaces , " " , nvertices)

298

In [205]:
EVlar, FElar = to_lar(vertices, faces)

check_vf(vertices, faces, true)
check_vevfe(vertices, EVlar, FElar)
visualize_numbers([vertices, EVlar, FElar], 2)

faces min 1, faces max 1192, vertex number 1192
1
5
9
13
17
21
25
29
33
37
41
45
49
53
57
61
65
69
73
77
81
85
89
93
97
101
105
109
113
117
121
125
129
133
137
141
145
149
153
157
161
165
169
173
177
181
185
189
193
197
201
205
209
213
217
221
225
229
233
237
241
245
249
253
257
261
265
269
273
277
281
285
289
293
297
301
305
309
313
317
321
325
329
333
337
341
345
349
353
357
361
365
369
373
377
381
385
389
393
397
401
405
409
413
417
421
425
429
433
437
441
445
449
453
457
461
465
469
473
477
481
485
489
493
497
501
505
509
513
517
521
525
529
533
537
541
545
549
553
557
561
565
569
573
577
581
585
589
593
597
601
605
609
613
617
621
625
629
633
637
641
645
649
653
657
661
665
669
673
677
681
685
689
693
697
701
705
709
713
717
721
725
729
733
737
741
745
749
753
757
761
765
769
773
777
781
785
789
793
797
801
805
809
813
817
821
825
829
833
837
841
845
849
853
857
861
865
869
873
877
881
885
889
893
897
901
905
909
913
917
921
925
929
933
937
941
945
949
953
957
961
965
969
973
977


In [130]:
full(x)

LoadError: UndefVarError: dict not defined

In [131]:
dctd = Dict()
dctd["aho"] = [5, 7]
dctd["ahoi"] = [3, 12]

dctd[tuple(11, 15)] = 5
dctd[tuple([12, 17]...)] = "yop"

println(dctd)
haskey(dctd, "aho")
println(haskey(dctd, (11, 15)))



dctd[(12,17)]


Dict{Any,Any}(Pair{Any,Any}("aho",[5,7]),Pair{Any,Any}((11,15),5),Pair{Any,Any}((12,17),"yop"),Pair{Any,Any}("ahoi",[3,12]))
true


"yop"

In [132]:
#blbost
EVinds = get_EVinds_naive(hexagon_faces)
FElar = get_FElar_naive(size(hexagon_faces, 1))

6×18 sparse matrix with 18 Int8 nonzero entries:
	[1 ,  1]  =  1
	[1 ,  2]  =  1
	[1 ,  3]  =  1
	[2 ,  4]  =  1
	[2 ,  5]  =  1
	[2 ,  6]  =  1
	[3 ,  7]  =  1
	[3 ,  8]  =  1
	[3 ,  9]  =  1
	[4 , 10]  =  1
	[4 , 11]  =  1
	[4 , 12]  =  1
	[5 , 13]  =  1
	[5 , 14]  =  1
	[5 , 15]  =  1
	[6 , 16]  =  1
	[6 , 17]  =  1
	[6 , 18]  =  1

## FElar

In [133]:
# face orientation
test_face1 = [1, 3, 5]
test_face2 = [3, 5, 1]



3-element Array{Int64,1}:
 3
 5
 1

In [134]:
# FE = get_FElar_naive(nfaces)
FEpl

LoadError: UndefVarError: FEpl not defined

In [135]:
# EVpl = collect(Eset)

In [136]:
function arrayofarray2arrayd2d(d)
    data = Array(Float64,length(d),length(d[1]))
    for i in 1:length(d)
        for j in 1:length(d[1])
            data[i,j] = d[i][j]
        end
    end
    return data
end

arrayofarray2arrayd2d (generic function with 1 method)

## Clean vertexes

In [137]:
oo = collect(zip([1,2,3],[4,5,6]))

3-element Array{Tuple{Int64,Int64},1}:
 (1,4)
 (2,5)
 (3,6)

In [138]:
collect(zip([10,2,3], range(1,3)))

3-element Array{Tuple{Int64,Int64},1}:
 (10,1)
 (2,2) 
 (3,3) 

In [139]:
sort(collect(zip([10,2,3], range(1,3))))

3-element Array{Tuple{Int64,Int64},1}:
 (2,2) 
 (3,3) 
 (10,1)

In [140]:

function conv2array_of_float_arrays(array2d)

    test_vertex_float = float(array2d)

    list_of_vertexes = Array[]
    for i in 1:size(test_vertex_float, 1)
        push!(list_of_vertexes, test_vertex_float[i,:])
    end
    return list_of_vertexes
end
    

conv2array_of_float_arrays (generic function with 1 method)

In [141]:
function remove_double_vertexes_alternative(V)
    """ Return reduced vertex list wiht search table.
    
    Alternativte removing of doublefaces. Hopefully more memory efficient
    """
    X = range(1, size(V,1))
    # Vs = [v for (v, x) in VIsorted]
    # Is = [x for (v, x) in VIsorted]
    Vs = []
#     Is = [0]*size(V, 1)
#     Is = zeros( Int64, size(test_vertex, 1))
    
    Is = Dict()

    prevv = nothing
    i = 1
    sorted_vertex_id_x = sortperm(collect(zip(test_vertex[:,1], test_vertex[:,2])))
               sortperm(collect(zip(test_vertex[:,1], test_vertex[:,2])))
#     for couple in sort(collect(zip(X, conv2array_of_float_arrays(V))))
    for x in sorted_vertex_id_x
        v = V[x,:]
#         x = 
#         x, v = couple
        println(v, prevv, v.==prevv)
        if v == prevv
            # prev index was increased
            Is[x] = i - 1
        else
            push!(Vs, v)
            Is[x] = i
            i = i + 1
            prevv = v
        end
    end
    
    vertex_reduced_list = Vs
    vertex_search_table = Is
    return vertex_reduced_list, vertex_search_table
end

remove_double_vertexes_alternative (generic function with 1 method)

In [142]:

# sortperm(collect(zip(test_vertex[:,1], test_vertex[:,2])))

In [143]:
vtfl = conv2array_of_float_arrays(test_vertex)
vtfl

5-element Array{Array,1}:
 [1.0,2.0]
 [2.0,3.0]
 [4.0,1.0]
 [2.0,3.0]
 [3.0,3.0]

In [144]:
[1 2 3 4] == [1 2 3 4]

true

In [145]:
sortperm([(6, 2), (2, 6), (1, 3)])

3-element Array{Int64,1}:
 3
 2
 1

## Construct EVlar


In [146]:
nedges = size(EVpl, 1)

LoadError: UndefVarError: EVpl not defined

In [147]:

EVlar = spzeros(Int8, nedges , nvertices )

for i = 1:nedges
    EVlar[i, EVpl[i][1]] = -1
    EVlar[i, EVpl[i][2]] = 1
end

EVlar


LoadError: UndefVarError: EVpl not defined

# FE

In [148]:
faces[1,2]

2

In [149]:
function get_FElar_naive(faces, nvertices=nothing, nfaces=nothing)
    if nfaces == nothing
        nfaces = size(faces,1)
    end
    if nvertices == nothing
        nvertices = maximum(faces)
    end
    # not oriented verison of faces
    FE = spzeros(Int8, nfaces, nvertices)
    for i = 1:nfaces
        FE[i, faces[i, 1]] = 1
        FE[i, faces[i, 2]] = 1
        FE[i, faces[i, 3]] = -1
    end
    
    return FE
end

get_FElar_naive (generic function with 3 methods)

In [150]:

function get_FElar(faces, nvertices=nothing, nfaces=nothing)
    if nfaces == nothing
        nfaces = size(faces,1)
    end
    if nvertices == nothing
        nvertices = maximum(faces)
    end
    FE = spzeros(Int8, nfaces, nvertices)
    for i = 1:nfaces
        sgn = face_orientation(faces[i, :])
        ordered_face = sort(faces[i, :])
        FE[i, ordered_face[1]] = sgn * 1
        FE[i, ordered_face[2]] = sgn * 1
        FE[i, ordered_face[3]] = sgn * -1
    end
    return FE
end


#test

get_FElar (generic function with 3 methods)

In [151]:

test_FElar = get_FElar(hexagon_faces)
# test_FElar[2 , 4]
full(test_FElar)
# test_FElar.nzval

6×7 Array{Int8,2}:
  1   1  -1   0   0   0  0
  0  -1  -1   1   0   0  0
  0  -1   0  -1   1   0  0
  0  -1   0   0  -1   1  0
  0  -1   0   0   0  -1  1
 -1  -1   0   0   0   0  1

In [152]:
vertices[7,:]
minimum(vertices)

0.0

In [153]:

visualize_numbers([vertices, EVlar, FElar], 2)

LoadError: BoundsError: attempt to access 0-element Array{Int64,1} at index [1]

# Debug version of visualize_numbers

In [154]:
function lar2plasm(EV, FE)
    EV = sparse(EV)
    faces = Array{Array{Int64, 1}, 1}()

    for f in 1:size(FE, 1)

        verts = Array{Int64, 1}()
        done = Array{Int64, 1}()
        face = FE[f, :]
        edges = EV[face.nzind, :]

        push!(done, 1)
        vs = edges[1, :].nzind
        if face.nzval[1] < 0
            vs = vs[end:-1:1]
        end
        startv, nextv = vs

        println("startv ", startv)
        push!(verts, startv)
        while nextv != startv
            println("ve while", edges[:, nextv].nzind, done, nextv, edges)
            ednz = edges[:, nextv].nzind
            println("wh 1")
            eall = setdiff(ednz, done)
            println("wh 2", eall)
            e = eall[1]
            println("wh 3")
            push!(done, e)
            vs = edges[e, :].nzind
            if face.nzval[e] < 0
                vs = vs[end:-1:1]
            end
            curv, nextv = vs
            push!(verts, curv)
            println("wh 5 ", nextv)
        end

        push!(faces, verts)
    end

    "FV = " * string(faces)[15:end]
end

function lar2py(V, EV)
    text = "V = [[0,0]"
    for i in 1:size(V, 1)
        text = text * "," * string(vec(V[i, :]))
    end
    text = text * "]\n"
    
    text = text * "EV = ["
    for i in 1:size(EV, 1)
        if i > 1 text = text * "," end
        text = text * string(EV[i, :].nzind)
    end
    text = text * "]"
    return text
end


function visualize_numbers(set, font_size)
    """
    seg = [V, EV, FE]
    """

    f = open("tmp.py", "w")

    file_content = "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size))\n"
    print(file_content)
    write(f, file_content)

    close(f)
#     run(`python tmp.py`)
end

function visualize_boundary(set, boundary, font_size)
    f = open("tmp.py", "w")
    file_content =  "from larlib import *\n" *
        lar2py(set[1], set[2]) * "\n" *
        lar2plasm(set[2], set[3]) * "\n" *
        "VV = AA(LIST)(range(len(V)))\n" *
        "submodel = STRUCT(MKPOLS((V,EV)))\n" *
        "model = larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,$font_size)\n" *
        lar2py(set[1], set[2][boundary.nzind, :]) * "\n" *
        "overmodel = STRUCT(MKPOLS((V,EV)))\n" *
        "VIEW(STRUCT([model, COLOR(RED)(overmodel)]))"
    print(file_content)
    write(f, file_content)

    close(f)
    run(`python tmp.py`)
end

visualize_boundary (generic function with 1 method)

In [155]:
test_V = [
    1 1
    2 1
    1 2
    2 2
    ]
test_EV = sparse([
    -1 1 0 0
    0 -1 1 0
    -1 0 1 0
])
test_FE = sparse([
        1 1 -1
        -1 -1 1
])
visualize_numbers([test_V, test_EV, test_FE], 2)

startv 1
ve while[1,2][1]2
	[1, 1]  =  -1
	[3, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[2, 3]  =  1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5 3
ve while[2,3][1,2]3
	[1, 1]  =  -1
	[3, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[2, 3]  =  1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5 1
startv 2
ve while[1,3][1]1
	[1, 1]  =  -1
	[3, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[2, 3]  =  1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5 3
ve while[2,3][1,3]3
	[1, 1]  =  -1
	[3, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[2, 3]  =  1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5 2
from larlib import *
V = [[0,0],[1,1],[2,1],[1,2],[2,2]]
EV = [[1,2],[2,3],[1,3]]
FV = [[1,2,3],[2,1,3]]
VV = AA(LIST)(range(len(V)))
submodel = STRUCT(MKPOLS((V,EV)))
VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,2))


In [156]:
function printarr(arr)
    print(size(arr), typeof(arr), " sparse: ", issparse(arr),", min: ",  minimum(arr), ", max: ",  maximum(arr))
end

printarr (generic function with 1 method)

In [157]:
nvertices

1192

In [158]:
nfaces

298

In [159]:
printarr(vertices)

(1192,3)Array{Float64,2} sparse: false, min: 0.0, max: 6.0

In [160]:
printarr(EVlar)
# EVlar.nzind

(12,1192)SparseMatrixCSC{Int8,Int64} sparse: true, min: 0, max: 0

In [161]:
printarr(FElar)

(6,18)SparseMatrixCSC{Int8,Int64} sparse: true, min: 0, max: 1

In [162]:
vertices

1192×3 Array{Float64,2}:
 1.0  2.0  1.0
 2.0  2.0  1.0
 1.0  3.0  1.0
 2.0  3.0  1.0
 1.0  2.0  1.0
 2.0  2.0  1.0
 1.0  2.0  2.0
 2.0  2.0  2.0
 1.0  2.0  1.0
 1.0  3.0  1.0
 1.0  2.0  2.0
 1.0  3.0  2.0
 2.0  2.0  1.0
 ⋮            
 4.0  6.0  5.0
 5.0  6.0  5.0
 4.0  6.0  6.0
 5.0  6.0  6.0
 4.0  4.0  6.0
 5.0  4.0  6.0
 4.0  5.0  6.0
 5.0  5.0  6.0
 4.0  5.0  6.0
 5.0  5.0  6.0
 4.0  6.0  6.0
 5.0  6.0  6.0

In [163]:
full(EVlar)

12×1192 Array{Int8,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [164]:
FElarint = sparse(Int8, FElar)
# full(FElarint)

LoadError: MethodError: no method matching sparse(::Type{Int8}, ::SparseMatrixCSC{Int8,Int64})[0m
Closest candidates are:
  sparse(::Any, ::Any, [1m[31m::Number[0m) at sparse/sparsematrix.jl:587
  sparse(::Any, ::Any, [1m[31m::AbstractArray{Bool,1}[0m, [1m[31m::Any[0m, [1m[31m::Any[0m) at sparse/sparsematrix.jl:595
  sparse(::Any, ::Any, [1m[31m::AbstractArray{T,1}[0m) at sparse/sparsematrix.jl:589
  ...[0m

In [165]:

visualize_numbers([vertices, EVlar, FElar], 2)

LoadError: BoundsError: attempt to access 0-element Array{Int64,1} at index [1]

In [166]:
setdiff([2,3],[1,2,3])

0-element Array{Int64,1}

In [167]:
V = [
    1 1
    2 1
    1 2
    2 2
    ]
EV = sparse([
    1 -1 0 0
    0 1 -1 0
    -1 0 1 0
])
FE = sparse([
        1 1 -1
        -1 -1 1
])
visualize_numbers([V, EV, FE], 2)

startv 1
ve while[1,2][1]2
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5 3
ve while[2,3][1,2]3
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5 1
startv 2
ve while[1,3][1]1
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[3]
wh 3
wh 5 3
ve while[2,3][1,3]3
	[1, 1]  =  1
	[3, 1]  =  -1
	[1, 2]  =  -1
	[2, 2]  =  1
	[2, 3]  =  -1
	[3, 3]  =  1
wh 1
wh 2[2]
wh 3
wh 5 2
from larlib import *
V = [[0,0],[1,1],[2,1],[1,2],[2,2]]
EV = [[1,2],[2,3],[1,3]]
FV = [[1,2,3],[2,1,3]]
VV = AA(LIST)(range(len(V)))
submodel = STRUCT(MKPOLS((V,EV)))
VIEW(larModelNumbering(1,1,1)(V,[VV,EV,FV],submodel,2))


# Test visualization using LAR and Plasm

In [168]:
V = [
    1 1
    2 1
    1 2
    2 2
    ]
EV = sparse([
    1 -1 0 0
    0 1 -1 0
    -1 0 1 0
])
FE = sparse([
        1 1 -1
        -1 -1 1
])

2×3 sparse matrix with 6 Int64 nonzero entries:
	[1, 1]  =  1
	[2, 1]  =  -1
	[1, 2]  =  1
	[2, 2]  =  -1
	[1, 3]  =  -1
	[2, 3]  =  1

In [169]:
#visualize_numbers([V, EV, FE], 2)

In [170]:
function reindexVertexesInFaces(faces, new_indexes,
                                       index_base=0)
#     println(faces)
    for i in 1:size(faces, 1)
        for j in 1:size(faces, 2)
            faces[i, j] = new_indexes[faces[i, j] - index_base] + index_base
        end
    end
#     try
#     catch
#         println("reindexVertexInFaces exception")
#         println("fc ", face, " i ", i , " len ", size(new_indexes))
        
#     end
    return faces
    
end
    
        
#     for face in faces:
#         try:
#             for i in range(0, len(face)):
#                 face[i] = new_indexes[face[i] - index_base] + index_base
#             # face[0] = new_indexes[face[0]-1] + 1
#             # face[1] = new_indexes[face[1]-1] + 1
#             # face[2] = new_indexes[face[2]-1] + 1
#         except:
#             import traceback
#             traceback.print_exc()
#             print 'fc ', face, ' i ', i
#             print len(new_indexes)

#     return faces 
function removeDoubleFacesByDictAlgoritm(FW, sort_faces=true)
#     FWdict = Dict()
    set = Set()
    for k in 1:size(FW, 1)
        println(k)
        if sort_faces
            face = tuple(sort(FW[k, :])...)
        else
            face = tuple(FW[k, :]...)
        end
            
        push!(set, face)
    end
    return set
end
    

# def removeDoubleFacesByAlberto(FW):
#     from collections import defaultdict
#     # use of Albertos algorithm

#     cellDict = defaultdict(list)
#     for k, cell in enumerate(FW):
#         cellDict[tuple(cell)] += [k]
#     FW = [list(key) for key in cellDict.keys() if len(cellDict[key]) == 1]
#     return FW
function removeDoubleVertexesAndFaces(vertexes, faces, index_base=0)
    println("step1")
    new_vertexes, inv_vertexes = remove_double_vertexes_alternative(vertexes)# , index_base=index_base)
    println("step2")
    new_faces = reindexVertexesInFaces(faces, inv_vertexes)
    println("step3")
    new_faces = removeDoubleFacesByDictAlgoritm(faces)
    return new_vertexes, new_faces
end

removeDoubleVertexesAndFaces (generic function with 2 methods)

In [171]:
Dict(tuple(1,2) => 1, "B" => 2)

Dict{Any,Int64} with 2 entries:
  "B"   => 2
  (1,2) => 1

In [172]:
removeDoubleVertexesAndFaces()

LoadError: MethodError: no method matching removeDoubleVertexesAndFaces()[0m
Closest candidates are:
  removeDoubleVertexesAndFaces([1m[31m::Any[0m, [1m[31m::Any[0m) at In[170]:61
  removeDoubleVertexesAndFaces([1m[31m::Any[0m, [1m[31m::Any[0m, [1m[31m::Any[0m) at In[170]:61[0m

# Unittest

In [173]:
# test
test_vertex = [ 1 2; 2 3; 4 1; 2 3; 3 3]
test_faces = [
    1 3 4;
    4 5 1;
    1 5 4
]
# test_EV = sparse([
#     1 -1 0 0
#     0 1 -1 0
#     -1 0 1 0
# ])
# test_FE = sparse([
#         1 1 -1
#         -1 -1 1
# ])
# test_faces = sparse([ 1 2; 2 3; 4 1; 2 3])

Vs, Is = remove_double_vertexes_alternative(test_vertex)

for i in 1:size(test_vertex, 1)
    v1 = Vs[Is[i]] 
    v2 = test_vertex[i,:]
    assert(v1 == v2)
end

print(Vs)

[1,2]nothingBool[false,false]
[2,3][1,2]Bool[false,false]
[2,3][2,3]Bool[true,true]
[3,3][2,3]Bool[false,true]
[4,1][3,3]Bool[false,false]
Any[[1,2],[2,3],[3,3],[4,1]]

In [174]:

reindexed_faces = reindexVertexesInFaces(copy(test_faces), Is)
println(Vs)
println(reindexed_faces)
reindexed_faces == [1 4 2; 2 3 1]


Any[[1,2],[2,3],[3,3],[4,1]]
[1 4 2; 2 3 1; 1 3 2]


false

In [175]:
fcs = removeDoubleFacesByDictAlgoritm(test_faces)

collect(fcs)

1
2
3


2-element Array{Any,1}:
 (1,4,5)
 (1,3,4)

In [176]:

new_vertexes, new_faces = removeDoubleVertexesAndFaces(test_vertex, test_faces)

step1
[1,2]nothingBool[false,false]
[2,3][1,2]Bool[false,false]
[2,3][2,3]Bool[true,true]
[3,3][2,3]Bool[false,true]
[4,1][3,3]Bool[false,false]
step2
step3
1
2
3


(Any[[1,2],[2,3],[3,3],[4,1]],Set(Any[(1,2,3),(1,2,4)]))